In [1]:
import pandas as pd
import math
import numpy as np
import string
import nltk 
import re 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [2]:
df = pd.read_csv('../data/reddit_binary-jing-qian/reddit.csv')
df

,id,text,hate_speech_idx,response
0,1. e8q18lf\r\n2. \te8q9w5s\r\n3. \t\te8qbobk\r...,1. A subsection of retarded Hungarians? Ohh bo...,[1],"[""I don't see a reason why it's okay to insult..."
1,1. e9c6naz\r\n2. \te9d03a5\r\n3. \t\te9d8e4d\r\n,"1. > ""y'all hear sumn?"" by all means I live i...",[3],['Persons with disabilities is the accepted te...
2,1. e84rl2i\r\n2. \te84w60l\r\n3. \t\te8544rn\r...,1. wouldn't the defenders or whatever they are...,NaN,NaN
3,1. e7kq72n\r\n2. \te7m24ar\r\n,1. Because the Japanese aren't retarded and kn...,[1],"[""It's not right for anyone of any gender to b..."
4,1. e7hdgoh\r\n2. \te7iyj6a\r\n3. \t\te7j6iho\r...,1. That might be true if we didn't have an exa...,"[2, 3]","[""You shouldn't be bringing up sensitive topic..."
...,...,...,...,...
5015,1. e90fta8\r\n2. \te90g8px\r\n3. \t\te90jlq5\r...,1. Who does she claim as *her people*?\r\n2. \...,[8],['its inappropriate to use the word faggot as ...
5016,1. e9b7mn2\r\n2. \te9bv6ir\r\n,1. Here's what's going to happen: Broward is ...,NaN,NaN
5017,1. e7q8ilv\r\n2. \te7qddqq\r\n3. \t\te7qstbc\r...,1. Oh boy...so here comes a long venting post ...,"[1, 4]","[""using the word 'nigger' is a direct attack t..."
5018,1. dwp8jut\r\n,"1. OP, stop being a faggot and post videos nex...",[1],"[""using the words 'faggot fuckers' is offensiv..."


In [3]:
hate_sents = []
for i in range(len(df.index)):
    hate_speech_idx = df['hate_speech_idx'][i]
    if type(hate_speech_idx) != float:
        temp_text = df['text'][i].split('\n')
        hate_speech_idx = hate_speech_idx.replace('[', '').replace(']', '').split(',')
        for idx in hate_speech_idx:
            if int(idx) < len(temp_text):
                hate_sents.append(temp_text[int(idx) - 1])
hate_sents = [sent.replace('\r', '').replace('\t', '') for sent in hate_sents]
hate_sents = [sent[3: ].replace('>', '').replace('\t', '') for sent in hate_sents]
hate_sents

['A subsection of retarded Hungarians? Ohh boy. brace for a livid Bulbasaur coming in here trying to hate a hole in some of her stupider countrymen.',
 'ah a liberal has slipped in. you can tell because the first question asked was loaded and retarded.    What does "working hard" mean? Your political views get more validated by how much manual labour you put in?   where did I say that? and yes, of course you have no clue what "working hard" means. Few soyboys do.   Based on your comment regarding the EC I guess you don\'t really think that 2 mil majority of people who voted for Hillary don\'t really matter.  I shouldn\'t have to explain to you the point of the EC. If you want to talk to someone who is more patient with you than I am, you can check out AskThe_Donald. ',
 "Because the Japanese aren't retarded and know that *fictional characters* aren't owed anything, much less dignity.",
 "Honestly, I believe that Kavanaugh probably did do what she says he did. Especially after more info

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
stop_words = set(stopwords.words('english')) 
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [6]:
vect = TfidfVectorizer(stop_words='english')
tfidf = vect.fit_transform(hate_sents)

In [7]:
print("TF-IDF vectors (each column is a document):\n{}\nRows:\n{}".format(tfidf.T.A, vect.get_feature_names()))

TF-IDF vectors (each column is a document):
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Rows:
['00', '000', '007', '00km', '01', '014', '02', '03', '04', '05', '06', '079020', '08', '09', '0bama0', '0k5z7vwjg58', '0mcm0fyytp8', '0tiig', '0xirq', '10', '100', '1000', '10000', '100000', '100000004471953', '100k', '100s', '100x', '101st', '1050830549089046528', '1050920033331933184', '1052013803930603521', '1052019471496474624', '1052185336019910657', '1052191063094509569', '1052284117411348480', '1052289455384653824', '1052372165167894528', '1052655359427575808', '1052678435695460353', '10927507', '10c', '10k', '10th', '11', '110mm', '1111', '11369', '113897', '114', '115', '119', '11e5f4296cec', '11e8', '12', '120', '1200', '12123', '125', '1269', '12pm', '12year', '13', '1374122269', '14', '1400', '1400s', '1423', '1427', '145th', '1488', '14th', '15', '150', '1500', '151', '

In [8]:
freqs = pd.Series(tfidf.T.A.sum(axis=1), index=vect.get_feature_names())
freqs

00             0.706859
000            3.889870
007            0.308300
00km           0.040839
01             0.601101
                 ...   
ünser          0.181635
ýou            0.260885
ಠ_ಠ            1.655342
杨怡潇krasa       0.313467
英語で全部書いて下さい    0.372691
Length: 16912, dtype: float64

In [9]:
freqs = freqs.sort_values(ascending=False)
freqs[: 50]

cunt        231.190838
retarded    187.487280
fucking     119.831130
like        117.742734
just        111.591297
retard      102.523627
people       93.520290
faggot       89.771177
don          88.635685
fuck         82.133470
cunts        79.293649
women        70.818327
shit         63.285937
think        59.761811
retards      58.857354
know         51.399982
men          51.295789
right        49.012239
white        45.296106
stupid       43.858251
say          42.435711
man          40.020215
nigger       39.856346
bitch        39.787780
dumb         39.359554
time         39.337422
really       37.929450
want         37.473020
actually     37.342172
good         36.797714
got          35.583499
lol          35.068137
way          35.016534
make         34.885031
did          34.852938
woman        34.183531
doesn        34.102398
ve           33.926205
faggots      31.379756
going        31.361337
person       30.536828
thing        29.862736
use          29.844564
said       